In [43]:
import sys
import os

# Get directory
current_dir = os.getcwd()

# Create project_root for module imports
project_root = os.path.abspath(os.path.join(current_dir, ".."))
sys.path.append(project_root)

# Create data directory path
parent_dir = os.path.dirname(current_dir)
save_dir = os.path.join(parent_dir, "final_messages", "n_body_gravity")
os.makedirs(save_dir, exist_ok=True)

# Import relevant modules
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch_geometric.loader import DataLoader
import torch
import random
from gnn_model.graph_structure_from_trajecotry import node_data_list
from gnn_model.message_passing_MLP import GNN_MLP
from gnn_model.train_model import train_model
from pysr import PySRRegressor

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/juliacall/__init__.py:61: UserWarning: torch was imported before juliacall. This may cause a segfault. To avoid this, import juliacall before importing torch. For updates, see https://github.com/pytorch/pytorch/issues/78829.
  warnings.warn(


Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [ ]:
def generate_random_positions(N, dim, min_dist, box_size):
    positions = []
    while len(positions) < N:
        pos = torch.rand(dim) * box_size
        if all(torch.norm(pos - p) >= min_dist for p in positions):
            positions.append(pos)
    return torch.stack(positions)

def generate_random_velocities(N, dim, velocity_scale=1.0):
    return (torch.rand((N, dim)) - 0.5) * 2 * velocity_scale

def compute_gravitational_forces(positions, masses, G=1.0, eps=1e-5):
    N, dim = positions.shape
    forces = torch.zeros_like(positions)
    for i in range(N):
        for j in range(i + 1, N):
            r_vec = positions[j] - positions[i]
            dist = torch.norm(r_vec) + eps
            force_mag = G * masses[i] * masses[j] / dist**2
            force_dir = r_vec / dist
            force = force_mag * force_dir
            forces[i] += force
            forces[j] -= force
    return forces

def generate_unique_masses(N, mass_range, resolution=25):

    # Create a fine grid in the range
    mass_grid = torch.linspace(mass_range[0], mass_range[1], resolution).tolist()
    # Randomly choose N unique masses
    unique_masses = random.sample(mass_grid, N)

    return torch.tensor(unique_masses, dtype=torch.float32)

def n_body_simulation(N=5, T=100, dt=0.01, dim=2,
                      mass_range=(1.0, 30.0), min_dist=0.5,
                      box_size=10.0, velocity_scale=1.0, G=1.0):
    # Initialize
    masses = generate_unique_masses(N, mass_range)
    positions = generate_random_positions(N, dim, min_dist, box_size)
    velocities = generate_random_velocities(N, dim, velocity_scale)

    # Store results
    trajectory = torch.zeros((T, N, dim), dtype=torch.float32)
    trajectory_velocities = torch.zeros((T, N, dim), dtype=torch.float32)
    t_array  = torch.arange(0, T * dt, dt, dtype=torch.float32)

    for t in range(T):
        trajectory[t] = positions
        trajectory_velocities[t] = velocities

        # Compute forces and update positions & velocities (Euler method)
        forces = compute_gravitational_forces(positions, masses, G)
        accelerations = forces / masses[:, None]
        velocities = velocities + accelerations * dt
        positions = positions + velocities * dt

    trajectory_data = {
        "time": t_array,
        "positions": trajectory,
        "velocities": trajectory_velocities,
        "masses": masses
    }

    return trajectory_data



In [39]:

from sklearn.model_selection import train_test_split
import pandas as pd

def run_pipeline(iterations=10, train_fraction=0.7,
                 N=5, T=100, dt=0.01, dim=2, hidden_channels=64,
                 m_dim=2, out_channels=2, epochs=100, lr=0.001):

    # 1) Run simulations
    all_trajectories = [n_body_simulation(N=N, T=T, dt=dt, dim=dim) for _ in range(iterations)]

    # 2) Convert to PyG data objects
    all_graph_data = []
    dataset_index = []
    for idx, traj in enumerate(all_trajectories):
        graphs = node_data_list(traj, self_loop=False, complete_graph=True)
        all_graph_data.extend(graphs)
        dataset_index.extend([idx] * len(graphs))

    # 3) Split into train/test
    indices = list(range(len(all_graph_data)))
    train_idx, test_idx = train_test_split(indices, train_size=train_fraction, stratify=dataset_index)

    train_data = [all_graph_data[i] for i in train_idx]
    test_data = [all_graph_data[i] for i in test_idx]

    # 4) Initialize the model
    input_dim = train_data[0].x.shape[1]
    model = GNN_MLP(n_f=input_dim, m_dim=m_dim, hidden_channels=hidden_channels,
                    out_channels=out_channels, single_node=False)

    # 5) Train model (save messages from final epoch)
    model = train_model(model, train_data, epochs=epochs, lr=lr)

    # 6) Extract training messages
    train_messages = pd.DataFrame(model.message_storage)
    train_messages[['pos_i_x', 'pos_i_y']] = pd.DataFrame(train_messages['pos_i'].tolist())
    train_messages[['pos_j_x', 'pos_j_y']] = pd.DataFrame(train_messages['pos_j'].tolist())
    train_messages[['message_x', 'message_y']] = pd.DataFrame(train_messages['message'].tolist())
    train_messages = train_messages.drop(columns=['pos_i', 'pos_j', 'message', 'edge'])

    # 7) Test on held-out data and store messages
    model.message_storage = []  # Reset
    for data in test_data:
        _ = model(data.x, data.edge_index, save_messages=True)

    test_messages = pd.DataFrame(model.message_storage)
    test_messages[['pos_i_x', 'pos_i_y']] = pd.DataFrame(test_messages['pos_i'].tolist())
    test_messages[['pos_j_x', 'pos_j_y']] = pd.DataFrame(test_messages['pos_j'].tolist())
    test_messages[['message_x', 'message_y']] = pd.DataFrame(test_messages['message'].tolist())
    test_messages = test_messages.drop(columns=['pos_i', 'pos_j', 'message', 'edge'])

    return model, train_messages, test_messages


In [49]:
from sklearn.model_selection import train_test_split
import pandas as pd

def run_pipeline_multi(train_iterations=100, test_iterations=40,
                 N_train=2, N_test_list=[3, 4, 5, 6], T=500, dt=0.01, dim=2, hidden_channels=128,
                 m_dim=2, out_channels=2, epochs=200, lr=0.001):

    # 1) Run training simulations with N_train
    train_trajectories = [n_body_simulation(N=N_train, T=T, dt=dt, dim=dim) for _ in range(train_iterations)]

    # 2) Convert training data to PyG objects
    # all_train_graph_data = []
    # train_dataset_index = []
    # for idx, traj in enumerate(train_trajectories):
    #     graphs = node_data_list(traj, self_loop=False, complete_graph=True)
    #     all_train_graph_data.extend(graphs)
    #     train_dataset_index.extend([idx] * len(graphs))

    # # 3) Split training data
    # train_indices, _ = train_test_split(
    #     list(range(len(all_train_graph_data))),
    #     train_size=train_fraction,
    #     stratify=train_dataset_index
    # )
    
    
    train_graph_data = []
    for traj in train_trajectories:
        graphs = node_data_list(traj, self_loop=False, complete_graph=True)
        train_graph_data.extend(graphs)
    
    # train_data = [all_train_graph_data[i] for i in train_indices]
    train_data = [train_graph_data[i] for i in range(len(train_graph_data))]

    # 4) Initialize model
    input_dim = train_graph_data[0].x.shape[1]
    model = GNN_MLP(n_f=input_dim, m_dim=m_dim, hidden_channels=hidden_channels,
                    out_channels=out_channels, single_node=False)

    # 5) Train model
    model = train_model(model, train_data, epochs=epochs, lr=lr)

    # 6) Extract training messages
    train_messages = pd.DataFrame(model.message_storage)
    train_messages[['pos_i_x', 'pos_i_y']] = pd.DataFrame(train_messages['pos_i'].tolist())
    train_messages[['pos_j_x', 'pos_j_y']] = pd.DataFrame(train_messages['pos_j'].tolist())
    train_messages[['message_x', 'message_y']] = pd.DataFrame(train_messages['message'].tolist())
    train_messages = train_messages.drop(columns=['pos_i', 'pos_j', 'message', 'edge'])

    # 7) Run and store test messages for each N in N_test_list
    test_messages_all = {}
    for N_test in N_test_list:
        test_trajectories = [n_body_simulation(N=N_test, T=T, dt=dt, dim=dim) for _ in range(test_iterations)]
        test_graph_data = []
        for traj in test_trajectories:
            graphs = node_data_list(traj, self_loop=False, complete_graph=True)
            test_graph_data.extend(graphs)

        model.message_storage = []
        for data in test_graph_data:
            _ = model(data.x, data.edge_index, save_messages=True)

        test_messages = pd.DataFrame(model.message_storage)
        test_messages[['pos_i_x', 'pos_i_y']] = pd.DataFrame(test_messages['pos_i'].tolist())
        test_messages[['pos_j_x', 'pos_j_y']] = pd.DataFrame(test_messages['pos_j'].tolist())
        test_messages[['message_x', 'message_y']] = pd.DataFrame(test_messages['message'].tolist())
        test_messages = test_messages.drop(columns=['pos_i', 'pos_j', 'message', 'edge'])

        test_messages_all[N_test] = test_messages

    return model, train_messages, test_messages_all


In [51]:
model_1, train_messages_1, test_messages_1 = run_pipeline_multi(train_iterations=100, test_iterations=20,
                 N_train=2, N_test_list=[3, 4, 5, 6], T=100, dt=0.01, dim=2, hidden_channels=128,
                 m_dim=2, out_channels=2, epochs=100, lr=0.001) 

/Users/jacobtoot/Documents/GitHub/DeepLearningPH/gnn_model/graph_structure_from_trajecotry.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_target = torch.tensor(acceleration, dtype=torch.float32)
/Users/jacobtoot/Documents/GitHub/DeepLearningPH/gnn_model/graph_structure_from_trajecotry.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_target = torch.tensor(acceleration, dtype=torch.float32)
/Users/jacobtoot/Documents/GitHub/DeepLearningPH/gnn_model/graph_structure_from_trajecotry.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTenso

In [42]:
train_messages_1.to_csv(f"{save_dir}/messages_cleaned.csv", index=False)

for i in range(3,8):
    test_messages_1[i].to_csv(f"{save_dir}/N_{i}_messages_test_cleaned.csv", index=False)

In [45]:
# Load your cleaned DataFrame
train_df = pd.read_csv(f"{save_dir}/messages_cleaned.csv")
test_df_3 = pd.read_csv(f"{save_dir}/N_3_messages_test_cleaned.csv")

train_df['dx'] = train_df['pos_j_x'] - train_df['pos_i_x']
train_df['dy'] = train_df['pos_j_y'] - train_df['pos_i_y']
train_df['distance'] = np.sqrt(train_df['dx']**2 + train_df['dy']**2)

test_df_3['dx'] = test_df_3['pos_j_x'] - test_df_3['pos_i_x']
test_df_3['dy'] = test_df_3['pos_j_y'] - test_df_3['pos_i_y']
test_df_3['distance'] = np.sqrt(test_df_3['dx']**2 + test_df_3['dy']**2)

# Define input features (X) and target (y)
# features = ['pos_i_x', 'pos_i_y', 'pos_j_x', 'pos_j_y', 'mass_i', 'mass_j', 'time', 'distance']
features = ['mass_i', 'mass_j', 'dx', 'dy','distance']

train_df['force_x'] = train_df['message_x'] * train_df['mass_i']
train_df['force_y'] = train_df['message_y'] * train_df['mass_i']

train_X = train_df[features]
train_y_x = train_df['message_x']
train_y_y = train_df['message_y']

test_df_3['force_x'] = test_df_3['message_x'] * test_df_3['mass_i']
test_df_3['force_y'] = test_df_3['message_y'] * test_df_3['mass_i']

test_X = test_df_3[features]
test_y_x = test_df_3['message_x']
test_y_y = test_df_3['message_y']

# TRAINING REGRESSION

In [48]:

# Create and fit SR model for message_x
train_model_x = PySRRegressor(
    niterations=100,
    binary_operators=["+", "-", "*", "/"],
    # unary_operators=["sin", "cos", "exp", "log"],
    model_selection="best",  # Select best tradeoff between complexity and error
    select_k_features=5,  # small number of features
    verbosity=1,
)

train_model_x.fit(train_X.values, train_y_x.values)

# Print best expression for message_x
print("Best expression for message_x:")
print(train_model_x)

# Optionally: model for message_y too
train_model_y = PySRRegressor(
    niterations=100,
    binary_operators=["+", "-", "*", "/"],
    # unary_operators=["sin", "cos", "exp", "log"],
    model_selection="best",
    select_k_features=5,  # small number of features
    verbosity=1,
)

train_model_y.fit(train_X.values, train_y_y.values)

print("Best expression for message_y:")
print(train_model_y)


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pysr/sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Using features ['x0' 'x1' 'x2' 'x3' 'x4']


[ Info: Started!
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pysr/sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Expressions evaluated per second: 2.330e+05
Head worker occupation: 14.9%
Progress: 486 / 1500 total iterations (32.400%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           5.206e+04  1.594e+01  y = -233.15
3           3.084e+04  2.617e-01  y = -13.363 * x4
5           2.168e+04  1.762e-01  y = -115.47 - (x4 * 10.222)
7           2.022e+04  3.494e-02  y = (-10.222 * x4) - (x3 - -115.48)
9           1.833e+04  4.904e-02  y = ((-10.221 * x4) + (x2 * 4.4387)) + -115.48
11          1.749e+04  2.348e-02  y = (-115.47 - (x4 * 10.222)) - ((x3 - x2) * 2.3733)
13          1.741e+04  2.206e-03  y = (-115.47 - (x4 * 10.222)) - (((x3 - x2) * 2.3733) - x2)
15          1.741e+04  9.209e-06  y = ((((((x2 * 0.69375) - x4) - x4) / 0.19918) - 117.55) - x3)...
                                   - x3
17          1.734e+04  2.169e-03  y = ((x2 - x3) - ((((x4 - -12.616) / 0.098322) - x2) - x

[ Info: Started!



Expressions evaluated per second: 2.360e+05
Head worker occupation: 17.8%
Progress: 470 / 1500 total iterations (31.333%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.217e+05  1.594e+01  y = -358.79
3           7.753e+04  2.255e-01  y = x4 * -20.211
5           5.437e+04  1.774e-01  y = (x4 * -15.217) + -183.6
7           5.224e+04  2.003e-02  y = (x4 * -15.217) + (-183.59 + x2)
9           4.664e+04  5.669e-02  y = ((13.029 + x4) / -0.067332) + (x2 * 6.7698)
11          4.613e+04  5.460e-03  y = ((x4 / -0.065821) - (x0 / (-1.8799 / x2))) - 184.67
13          4.609e+04  4.831e-04  y = (((x4 - -0.5304) / -0.065821) - ((x0 * -0.5304) * x2)) - 1...
                                  84.67
15          4.600e+04  9.494e-04  y = ((((x4 - -1.2735) / -0.065821) - (x2 * (x0 * -0.5304))) - ...
                                  184.67) + x0
17          4.551e+04  5.415e-03 

# TESTING REGRESSION

In [ ]:

# Create and fit SR model for message_x
test_model_x = PySRRegressor(
    niterations=100,
    binary_operators=["+", "-", "*", "/"],
    # unary_operators=["sin", "cos", "exp", "log"],
    model_selection="best",  # Select best tradeoff between complexity and error
    select_k_features=5,  # small number of features
    verbosity=1,
)

test_model_x.fit(test_X.values, test_y_x.values)

# Print best expression for message_x
print("Best expression for message_x:")
print(test_model_x)

# Optionally: model for message_y too
test_model_y = PySRRegressor(
    niterations=100,
    binary_operators=["+", "-", "*", "/"],
    # unary_operators=["sin", "cos", "exp", "log"],
    model_selection="best",
    select_k_features=5,  # small number of features
    verbosity=1,
)

test_model_y.fit(test_X.values, test_y_y.values)

print("Best expression for message_y:")
print(test_model_y)


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pysr/sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Using features ['x0' 'x1' 'x2' 'x3' 'x4']

Expressions evaluated per second: 1.270e+05
Head worker occupation: 9.7%
Progress: 274 / 1500 total iterations (18.267%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.459e+04  1.594e+01  y = -0.13383
3           1.725e+03  1.068e+00  y = -95.543 - x1
5           1.246e+03  1.627e-01  y = -116.95 - (x3 / x2)
7           1.008e+03  1.058e-01  y = -121.48 - ((x3 + x3) / x2)
9           8.704e+02  7.356e-02  y = -121.48 - (((x3 + x4) + x3) / x2)
10          8.364e+02  3.982e-02  y = (-112.8 - ((log(x0) / x2) * x3)) - x4
11          7.877e+02  5.995e-02  y = (-121.48 - (((x3 + x4) + x3) / x2)) + x2
12          7.653e+02  2.881e-02  y = ((-112.8 - x4) - ((log(x0) / x2) * x3)) + x2
13          7.299e+02  4.746e-02  y = ((-112.8 - (((x3 + x3) + x4) / x2)) + x2) - x4
14          7.282e+02  2.317e-03  y = ((-112.8 - ((x3 * (log(x0)

[ Info: Started!



Expressions evaluated per second: 1.270e+05
Head worker occupation: 9.0%
Progress: 523 / 1500 total iterations (34.867%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.459e+04  1.594e+01  y = -0.13383
2           1.455e+04  2.750e-03  y = sin(x4)
3           1.725e+03  2.132e+00  y = -95.543 - x1
5           1.246e+03  1.627e-01  y = -116.95 - (x3 / x2)
7           1.003e+03  1.084e-01  y = -121.48 - ((x3 / 0.46954) / x2)
8           8.963e+02  1.126e-01  y = -120.81 - ((log(x0) / x2) * x3)
9           8.673e+02  3.291e-02  y = -121.48 - ((x4 + (x3 / 0.43881)) / x2)
10          8.253e+02  4.966e-02  y = -120.81 - (((log(x0) / x2) * x3) - x2)
11          7.847e+02  5.048e-02  y = (-121.48 - ((x4 + (x3 / 0.43881)) / x2)) + x2
12          7.593e+02  3.284e-02  y = ((-116.06 - (x3 * (log(x0) / x2))) - x4) + x2
13          7.242e+02  4.733e-02  y = (-115.38 - ((((x3 / 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pysr/sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
[ Info: Started!



Expressions evaluated per second: 1.340e+05
Head worker occupation: 8.6%
Progress: 260 / 1500 total iterations (17.333%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
2           5.891e+04  7.971e+00  y = sin(-1.012)
3           1.003e+04  1.770e+00  y = -202.61 - x0
5           7.757e+03  1.285e-01  y = -220.12 - (x4 * x3)
7           7.043e+03  4.828e-02  y = (-202.61 - (x3 * x4)) - x0
8           6.233e+03  1.223e-01  y = -218.48 + ((x4 + x3) / cos(x1))
9           6.230e+03  3.587e-04  y = (-144.39 - (x4 * (x0 + x3))) + x1
10          5.723e+03  8.496e-02  y = (-214.91 + ((x3 + x4) / cos(x1))) - x0
12          5.344e+03  3.420e-02  y = (((x4 + x3) / cos(x1)) + (-218.56 - x0)) + x1
14          5.218e+03  1.197e-02  y = (x1 - x3) + ((-218.56 - x0) + ((x3 + x4) / cos(x1)))
15          4.962e+03  5.026e-02  y = (x1 + ((x3 / cos(912.74)) + (-218.48 - x0))) + (x3 / cos(x...
    